In [ ]:
import open3d as o3d
from sklearn import cluster

import read_pts

In [ ]:
pcd = o3d.io.read_point_cloud('point_cloud.pts')

In [ ]:
pcd_down = pcd.voxel_down_sample(voxel_size=0.001)  # 1mm
o3d.visualization.draw_geometries([pcd_down])

In [ ]:
df = read_pts.pcd_to_df(pcd_down)

In [ ]:
df_filter1 = df[(df['s'] < 0.05) & (df['v'] > 0.2)]
read_pts.write_pts('output/filter1.pts', df_filter1)
o3d.visualization.draw_geometries([read_pts.df_to_pcd(df_filter1)])

In [ ]:
df_filter2 = df[(df['s'] < 0.05) & (df['v'] > 0.2) & (df['z'] > 0.1)]
read_pts.write_pts('output/filter2.pts', df_filter2)
o3d.visualization.draw_geometries([read_pts.df_to_pcd(df_filter2)])

In [ ]:
pcd_filter2 = read_pts.df_to_pcd(df_filter2)
_, ind = pcd_filter2.remove_radius_outlier(nb_points=40, radius=0.005)
pcd_inliers = pcd_filter2.select_by_index(ind)
pcd_inliers.paint_uniform_color([0.7, 0.7, 0.7])
o3d.visualization.draw_geometries([pcd_inliers])

In [ ]:
model = cluster.KMeans(n_clusters=128, n_init='auto')
model.fit(pcd_inliers.points)

In [ ]:
clusters = o3d.geometry.PointCloud()
clusters.points = o3d.utility.Vector3dVector(model.cluster_centers_)
o3d.io.write_point_cloud('output/clusters.xyz', clusters, write_ascii=True)

In [ ]:
o3d.visualization.draw_geometries([pcd_down, clusters])